In [1]:
import os
import glob
import pandas as pd
import random
os.chdir("../../data/donors/")

In [2]:
wkdapath="wkda/*"
modulecolorpath="node500_merge.tsv"
nodepath="node_taxa500.tsv"
outpath="node_taxa500_new.tsv"

In [3]:
def changeurl(kd_module):
    kds_numb=kd_module.split(",")
    kds_color=kd_module.split(",")
    for i in range(len(kds_numb)):
        kds_numb[i]="1"
    
    for i in range(len(kds_color)):
        color=kds_color[i].split("_")[0]
        try:
            kds_color[i]=color2hex[color].replace("#","")
        except KeyError:
            kds_color[i]=color2hex["gray"].replace("#","")
    baseurl="https://chart.apis.google.com/chart?cht=p&chs=50x50&chf=bg,s,00000000&chd=t:%s&chco=%s" %(",".join(kds_numb),"|".join(kds_color))
    return (baseurl)


color2hex={"green":"#00FF00","grey":"#808080","turquoise":"#30D5C8",
          "blue":"#007bff","yellow":"#FFFF00","brown":"#964B00","darkgreen":"#013220","violet":"#8F00FF",
          "pink":"#FFC0CB","midnightblue":"#191970","orange":"#FFA500","black":"#000000","saddlebrown":"#8b4513",
          "paleturquoise":"#afeeee","lightcyan":"#e0ffff","yellowgreen":"#9ACD32","salmon":"#FA8072","white":"#FFFFFF",
          "skyblue":"#87CEEB","greenyellow":"#adff2f","magenta":"#FF00FF","darkmagenta":"#8b008b","tan":"#D2B48C",
          "sienna3":"#cd6839","grey60":"#999999","darkorange":"#ff8c00","lightyellow":"#ffffe0","darkturquoise":"#00ced1",
          "lightgreen":"#90ee90","red":"#ff0000","purple":"#a020f0","royalblue":"#4169e1","steelblue":"#4682b4",
          "darkgrey":"#a9a9a9","gray":"#808080","skyblue3":"#6ca6cd","darkred":"#8b0000","darkolivegreen":"#556b2f",
          "paleturquoise":"#afeeee","cyan":"#00ffff","mediumorchid":"#ba55d3","coral2":"#ee6a50","brown4":"#8b2323",
          "orangered4":"#8b2500","darkorange2":"#ee7600","lightpink4:":"#8b5f65","lightsteelblue1":"#cae1ff",
          "bisque4":"#8b7d6b","lightsteelblue":"#b0c4de","plum1":"#ffbbff","palevioletred3":"#cd6889","thistle2":"#eed2ee",
          "coral1":"#ff7256","lavenderblush3":"#cdc1c5","mediumpurple3":"#8968cd","skyblue2":"#7ec0ee"}

In [ ]:
for wkda_file in glob.glob(wkdapath):
    print(wkda_file)
    if("cytoscape_module_color_mapping.txt" in wkda_file):
        wkda_moudule_color=wkda_file
    elif("cytoscape_nodes.txt" in wkda_file):
        wdka_nodes=wkda_file
    elif("wKDA_kd_full_results.txt" in wkda_file):
        wdka_kd_full=wkda_file
module_color_mapping=pd.read_csv(wkda_moudule_color,sep="\t",index_col=0)
new_color=[]
for index,row in module_color_mapping.iterrows():
    module=row["MODULES"].split("_")[0]
    if module in color2hex.keys():
        new_color.append(color2hex[module])
    else:
        color="#cfcfcf"
        new_color.append(color)
        color2hex[module]=color
module_color_mapping.loc[:,"NEW_COLOR"]=new_color
module_color_mapping.to_csv(wkda_moudule_color,sep="\t")  

In [ ]:
#cytoscape node file from Mergeomics result
kd_nodes=pd.read_csv(wdka_nodes,sep="\t")
#node with WGNCA module color information
module_colors=pd.read_csv(modulecolorpath,sep="\t",index_col=1)
#Original node file 
nodes=pd.read_csv(nodepath,sep="\t")
#read keydriver full result file from Mergeomics result
kds=pd.read_csv(wdka_kd_full,sep="\t")


#Add new column to original node file; Mergeomics' result will be integrated here
nodes["URL"]=""
nodes["MODULE"]=""
nodes["KD_MODULE"]=""

In [6]:
for index, row in nodes.iterrows():
    try:
        #column #7 is module 
        #second column will be COLOR
        module=module_colors.loc[row["NODE"]]["MODULE"]
        nodes.iloc[index,7]=module_colors.loc[row["NODE"]]["MODULE"]
        module_color=module.split("_")[0]

        nodes.iloc[index,2]=color2hex[module_color]
    except KeyError:
        nodes.iloc[index,7]="gray"
        nodes.iloc[index,2]=color2hex["gray"]        

In [7]:
kd=kd_nodes[kd_nodes["SHAPE"]=="Diamond"]
kd=kd.set_index("NODE")

#Integrate information from wKDA_cytoscape_nodes.txt file
for index, row in nodes.iterrows():
    try:
        node=row["NODE"]
        shape=kd.loc[node]["SHAPE"]
        kd_module=kd.loc[node]["KD_MODULE"]
        url=changeurl(kd_module)
        nodes.iloc[index,6]=url
        nodes.iloc[index,4]=shape
        nodes.iloc[index,8]=kd_module
    except:
        continue
        
#Integrate information from wKDA_kd_full_results.txt
key_drivers={}
for index,row in kds.iterrows():
    key=row["NODE"]
    if key not in key_drivers.keys():
        key_drivers[key]=[]
    key_drivers[key].append(row["MODULE"])
        

for index, row in nodes.iterrows():
    try:
        node=row["NODE"]
        shape="ellipse"
        url=""
        if node in key_drivers.keys():
            shape="diamond"
            url=changeurl(",".join(key_drivers[node]))
        
        nodes.iloc[index,6]=url
        nodes.iloc[index,4]=shape
#         nodes.iloc[index,8]=kd.loc[node]["KD_MODULE"]
        nodes.iloc[index,8]=",".join(key_drivers[node])
    except:
        continue

In [8]:
nodes.to_csv(outpath,sep="\t",index=False)